![title](Chapter-2.png "Header")
___
# Chapter 2 -  Machine Learning Based Recommendation Systems
## Segment 2 - Model-based Collaborative Filtering Systems
## SVD Matrix Factorization

In [ ]:
import numpy as np
import pandas as pd

import sklearn
from sklearn.decomposition import TruncatedSVD

The MovieLens dataset was collected by the GroupLens Research Project at the University of Minnesota. You can download the dataset for this demostration at the following URL: https://grouplens.org/datasets/movielens/100k/

### Preparing the data

In [ ]:
columns = ['user_id', 'item_id', 'rating', 'timestamp']
frame = pd.read_csv('u.data', sep='\t', names=columns)
frame.head()

In [ ]:
columns = ['item_id', 'movie title', 'release date', 'video release date', 'IMDb URL', 'unknown', 'Action', 'Adventure',
          'Animation', 'Childrens', 'Comedy', 'Crime', 'Documentary', 'Drama', 'Fantasy', 'Film-Noir', 'Horror',
          'Musical', 'Mystery', 'Romance', 'Sci-Fi', 'Thriller', 'War', 'Western']

movies = pd.read_csv('u.item', sep='|', names=columns, encoding='latin-1')
movie_names = movies[['item_id', 'movie title']]
movie_names.head()

In [ ]:
combined_movies_data = pd.merge(frame, movie_names, on='item_id')
combined_movies_data.head()

In [ ]:
combined_movies_data.groupby('item_id')['rating'].count().sort_values(ascending=False).head()

In [ ]:
filter = combined_movies_data['item_id']==50
combined_movies_data[filter]['movie title'].unique()

### Building a Utility Matrix

In [ ]:
rating_crosstab = combined_movies_data.pivot_table(values='rating', index='user_id', columns='movie title', fill_value=0)
rating_crosstab.head()

### Transposing the Matrix

In [ ]:
rating_crosstab.shape

In [ ]:
X = rating_crosstab.T
X.shape

### Decomposing the Matrix

In [ ]:
SVD = TruncatedSVD(n_components=12, random_state=17)

resultant_matrix = SVD.fit_transform(X)

resultant_matrix.shape

In [ ]:
resultant_matrix

### Generating a Correlation Matrix

In [ ]:
corr_mat = np.corrcoef(resultant_matrix)
corr_mat.shape

### Isolating Star Wars From the Correlation Matrix

In [ ]:
movie_names = rating_crosstab.columns
movies_list = list(movie_names)

star_wars = movies_list.index('Star Wars (1977)')
star_wars

In [ ]:
corr_star_wars = corr_mat[1398,:]
corr_star_wars.shape

### Recommending a Highly Correlated Movie

In [ ]:
list(movie_names[(corr_star_wars<1.0) & (corr_star_wars > 0.9)])


In [ ]:
list(movie_names[(corr_star_wars<1.0) & (corr_star_wars > 0.95)])